# Resumo

Este trabalho trata da preparação e disponibilização de uma série temporal de dados relacionados ao tema de queimadas de uma região do estado de São Paulo. Os seguintes dados do ano de 2016 serão utilizados:

|  #  | Dado | Detalhes | Formato | Res. Temp. | Res. Espacial | SRS | Serviços | 
|:----|:-----|:------ --|:-------:|:----------:|:-------------:|:---:|:--------:|
| 1 | Imagens Brutas Landsat8 | Órbita/Ponto 221/074, Bandas 1-11 | Tif | 16 dias (23 cenas) \* | 7721x7841| EPSG 4326 | WTSS/WMS |
| 2 | Índices NDVI, NBRL, DNDVI e DNBRL  | Órbita/Ponto 221/074 | Tif | 16 dias (23 cenas) \* | 6676x6053 | EPSG 4326 | WTSS/WMS |
| 3 | Máscara de Núvens | Órbita/Ponto 221/074 | Tif | 16 dias (23 cenas)\* | 6676x6053 | EPSG 4326 | WTSS/WMS |
| 4 | Reflectância Landsat8 | Órbita/Ponto 221/074, Bandas 2-7 | Tif | 16 dias (23 cenas)\* | 6676x6053 | EPSG 4326 | WTSS/WMS |
| 5 | Focos de Queimadas | 49721772 registros | Dump BD PostgreSQL | Dado do dia 2017-08-11 | - | - | WS próprio |
| 6 | Talhões | 34560 registros | Shapefile | Dado do dia 2017-08-11 | - | EPSG 4326| WFS |
| 7 | Temperatura e Umidade | -  | NetCDF | 1 dia (366 cenas) | 1200x1400 | - | WTSS/WMS |
| 8 | Risco de Fogo | -  | NetCDF | 1 dia (306* cenas) | 1200x1400 | - | WTSS/WMS |
| 9 | Precipitação | - | NetCDF | 1 dia (366 cenas) | 1200x1400 | - | WTSS/WMS |


** \* Datas**
>   2016-01-11, 2016-01-27, 2016-02-12, 2016-02-28, 2016-03-15, 2016-03-31, 2016-04-16, 2016-05-02, 2016-05-18, 2016-06-03, 2016-06-19, 2016-07-05, 2016-07-21, 2016-08-06, 2016-08-22, 2016-09-07, 2016-09-23, 2016-10-09, 2016-10-25, 2016-11-10, 2016-11-26, 2016-12-12, 2016-12-28

Além da preparação e disponibilização dos dados através de serviços web, será produzido um Notebook Python (ipynb) ilustrando o acesso e uso dos dados e uma análise dos dados.

 

# 1. Preparando os dados
As subseções a seguir trazem detalhes sobre a preparação de cada conjunto de dados:


### 1.1. Imagens Brutas Landsat8

Os nomes dos arquivos segue o formato ([Fonte](https://landsat.usgs.gov/what-are-naming-conventions-landsat-scene-identifiers)):

LXSPPPRRRYYYYDDDGSIVV_BB.tif:
- L: Landsat
- X: Sensor
- S: Satellite
- PPP: WRS path
- RRR: WRS row
- YYYY: Year
- DDD: Julian day of year
- GSI: Ground station identifier
- VV: Archive version number
- BB: Band name



### 1.2. 	Índices NDVI, NBRL, DNDVI e DNBRL

### 1.3. Máscara de Núvens

### 1.4. Reflectância Landsat8

### 1.5. Focos de Queimadas

### 1.6. Talhões

### 1.7. Temperatura e Umidade

Estes dados estão disponibilizados em formato netCDF com dois subsets em cada arquivo. Em parte dos dados (arquivos com nome no formato "**GPOSNMC**[20160302182016030218]**P.icn.TQ0213L042.nc**"), os subsets são *tems* e *umrs*. Para o outro grupo de arquivos (arquivos com nome no formato "**BAM**20161231182016123118**.nc**"), os subsets são *tk2m* e *ur2m*. 

Para extrair cada um dos subsets de cada tipo de arquivo, foram utilizados os seguintes comandos utilizando o *bash* do linux:


`find . -name "G*.nc" -exec gdal_translate NETCDF:'{}':tems tif/'{}'.tk2m.tif \;`

`find . -name "G*.nc" -exec gdal_translate NETCDF:'{}':umrs tif/'{}'.ur2m.tif \;`

`find . -name "B*.nc" -exec gdal_translate NETCDF:'{}':tk2m tif/'{}'.tk2m.tif \;`

`find . -name "B*.nc" -exec gdal_translate NETCDF:'{}':ur2m tif/'{}'.ur2m.tif \;`


Para padronizar os nomes dos arquivos, foi feito:

`find . -name "G*.tif" -exec  sh -c 'A=$(basename $1 | cut -c 8-15);B=$(basename $1 | cut -c 44-); C="${A}${B}"; mv $1 $C' - {} \;`


`find . -name "BAM*.tif" -exec  sh -c 'A=$(basename $1 | cut -c 4-11);B=$(basename $1 | cut -c 24-35); C="${A}${B}"; mv $1 $C' - {} \;`


### 1.8. Risco de Fogo
Os dados de risco de fogo estavam em formato netCDF, foram convertidos usando o seguinte comando:

find . -name "*.nc" -exec gdal_translate '{}' tif/'{}'.tif \;

### 1.9. Precipitação
Os dados de precipitação estavam em formato netCDF, foram convertidos usando o seguinte comando:

find . -name "*.nc" -exec gdal_translate '{}' tif/'{}'.tif \;

## 2. Verificando metadados 

Para a verificação dos metadados, foi utilizado os módulos ogr, osr e gdal da osgeo.
Para suportar esses módulos é preciso instalar a *gdal* e o *numpy*, usando os comandos:

`conda install gdal`

`conda upgrade numpy`


Fora implementadas duas funcoes para a extração dos metadados, **printRastaerInfo** e **printVectorInfo** para imprimir metadados de arquivos raster e vetoriais, respectivamente.

In [2]:
from osgeo import ogr, osr, gdal

# Enable python exceptions 
gdal.UseExceptions()


def printRasterInfo(fileName):
    src = gdal.Open(fileName)
    if src is None:
        print("Unable to open {}".format(fileName))

    print "[ COLS ] = {}".format(src.RasterXSize)
    print "[ ROWS ] = {}".format(src.RasterYSize)
    prj = src.GetProjection()
    
    if prj != None:
        print osr.SpatialReference(wkt=prj)
    
    for band in range(1, src.RasterCount+1 ):
        
        print "\n[ GETTING BAND ]: {}".format(band)
    
        srcband = src.GetRasterBand(band)

        stats = srcband.GetStatistics( True, True )

        print "\t[ STATS ] =  Minimum={}, Maximum={}, Mean={}, StdDev={}".format(stats[0],stats[1],stats[2],stats[3])

        print "\t[ NO DATA VALUE ] = {}".format(srcband.GetNoDataValue())
        print "\t[ MIN ] = {}".format(srcband.GetMinimum())
        print "\t[ MAX ] = {}".format(srcband.GetMaximum())
        print "\t[ SCALE ] = {}".format(srcband.GetScale())
        src = None
        
def printVectorInfo(fileName):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    dataSource = driver.Open(fileName, 0)
    # Check to see if shapefile is found.
    if dataSource is None:
        print "Could not open {}",format(fileName)
    else:
        layer = dataSource.GetLayer()
        print layer.GetSpatialRef()
        featureCount = layer.GetFeatureCount()
        print "[ NUMBER OF FEATURES] = {}".format(featureCount)
        feat = layer.GetNextFeature()
        geom = feat.GetGeometryRef()
        print "[ GEOMETRY NAME ] = {}".format(geom.GetGeometryName())
        
        
        layerDefinition = layer.GetLayerDefn()
        fmt = "{:10s} {:10s} {:5s}"
        print fmt.format("Name","Type", "Width")
        for i in range(layerDefinition.GetFieldCount()):
            fieldName =  layerDefinition.GetFieldDefn(i).GetName()
            fieldTypeCode = layerDefinition.GetFieldDefn(i).GetType()
            fieldType = layerDefinition.GetFieldDefn(i).GetFieldTypeName(fieldTypeCode)
            fieldWidth = layerDefinition.GetFieldDefn(i).GetWidth()
            GetPrecision = layerDefinition.GetFieldDefn(i).GetPrecision()

            print fmt.format(fieldName, fieldType, str(fieldWidth))  



In [3]:
# 2.1. Landsat 8
landSat8File = '/home/vconrado/Documents/dados/queimadas/221_074_2016/imagens_brutas/LC82210742016011LGN00_B1.TIF';
printRasterInfo(landSat8File)

[ COLS ] = 7721
[ ROWS ] = 7841
PROJCS["WGS 84 / UTM zone 22N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-51],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32622"]]

[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=0.0, Maximum=46816.0, Mean=17440.536482, StdDev=13497.3432714
	[ NO DATA VALUE ] = None
	[ MIN ] = 0.0
	[ MAX ] = 46816.0
	[ SCALE ] = 1.0


In [4]:
# 2.2. Índices
nbrl = '/home/vconrado/Documents/dados/queimadas/221_074_2016/indices/221_074_2016-01-11_nbrl.tif';
printRasterInfo(nbrl)

[ COLS ] = 6676
[ ROWS ] = 6053
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]

[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=0.154082298279, Maximum=0.814603626728, Mean=0.532789811162, StdDev=0.101695359567
	[ NO DATA VALUE ] = 0.0
	[ MIN ] = 0.154082298279
	[ MAX ] = 0.814603626728
	[ SCALE ] = 1.0


In [5]:
# 2.3. Nuvens
nuvens = '/home/vconrado/Documents/dados/queimadas/221_074_2016/nuvens/221_074_2016-01-11_nuvem.tif';
printRasterInfo(nuvens)

[ COLS ] = 6676
[ ROWS ] = 6053
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]

[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=1.0, Maximum=3.0, Mean=1.11773160393, StdDev=0.470746723086
	[ NO DATA VALUE ] = 0.0
	[ MIN ] = 1.0
	[ MAX ] = 3.0
	[ SCALE ] = 1.0


In [6]:
# 2.4. Reflectância
nuvens = '/home/vconrado/Documents/dados/queimadas/221_074_2016/reflectancia/221_074_2016-01-11_ref_B2.tif';
printRasterInfo(nuvens)

[ COLS ] = 6676
[ ROWS ] = 6053
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]

[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=0.0, Maximum=0.888872802258, Mean=0.320362609187, StdDev=0.253952737591
	[ NO DATA VALUE ] = None
	[ MIN ] = 0.0
	[ MAX ] = 0.888872802258
	[ SCALE ] = 1.0


In [7]:
# 2.5. Precipitação
prec = '/home/vconrado/Documents/dados/queimadas/precipitacao_cosch_2016/S10648241_201601011200.nc';
printRasterInfo(prec)

[ COLS ] = 1200
[ ROWS ] = 1400


[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=0.0, Maximum=90.6248016357, Mean=2.49681837593, StdDev=7.00832669619
	[ NO DATA VALUE ] = -9999.0
	[ MIN ] = 0.0
	[ MAX ] = 90.6248016357
	[ SCALE ] = None


In [8]:
# 2.6. Risco de Fogo
rf = '/home/vconrado/Documents/dados/queimadas/risco_fogo_2016/RF.20160101.nc';
printRasterInfo(rf)

[ COLS ] = 1200
[ ROWS ] = 1400


[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=0.0, Maximum=1.0, Mean=0.39165301621, StdDev=0.401821619298
	[ NO DATA VALUE ] = 9.96920996839e+36
	[ MIN ] = 0.0
	[ MAX ] = 1.0
	[ SCALE ] = None


In [9]:
# 2.7. Temperatura e Umidade Superfície
temp_us = '/home/vconrado/Documents/dados/queimadas/temperatura_umidade_superficie_2016/tif/BAM20161101182016110118.nc.tk2m.tif';
printRasterInfo(temp_us)

[ COLS ] = 1200
[ ROWS ] = 1400


[ GETTING BAND ]: 1
	[ STATS ] =  Minimum=269.818511963, Maximum=319.019714355, Mean=293.433673211, StdDev=10.6562843459
	[ NO DATA VALUE ] = -999.0
	[ MIN ] = 269.818511963
	[ MAX ] = 319.019714355
	[ SCALE ] = 1.0


In [10]:
# 2.8. Talhões
talhoes = '/home/vconrado/Documents/dados/queimadas/221_074_2016/talhoes/Mapa_Geral.shp';
printVectorInfo(talhoes)

GEOGCS["GCS_WGS_1984",
    DATUM["WGS_1984",
        SPHEROID["WGS_84",6378137,298.257223563]],
    PRIMEM["Greenwich",0],
    UNIT["Degree",0.017453292519943295]]
[ NUMBER OF FEATURES] = 34560
[ GEOMETRY NAME ] = POLYGON
Name       Type       Width
Bloco      String     50   
Talhão    String     50   
Unidade    String     50   
Contrato   String     254  


## 3. Carga dos dados

Para a carga dos dados, foi escolhido a construção de dois arrays com multiplos atributos por célula. O primeiro array possui os dados brutos e índices do LandSat8 e o segundo array os demais dados ambientais e de risco de queimada. 

Para fazer a carga dos dados, foi preciso converter os dados, juntar os diferentes atributos e reorganizar os dados do modo que o SciDB armazena internamento. Para isso, foram utilizados os seguinte programas/scripts:

**1.** [tiff-to-raw](https://github.com/tfogal/tiff-to-raw): Converte de Tiff para dados brutos (implementação de terceiro);

**2.**  [interleaver](https://github.com/vconrado/cap-386/tree/master/project/src/interleaver): Recebe os diferentes arquivos de atributos e junta na sequência esperada pelo SciDB (implementação própria);

**3.**  [chunkfier](https://github.com/vconrado/cap-386/tree/master/project/src/chunkfier): rearranja um cubo de dados no formato que é usado para armazenamento pelo SciDB. Reduz o tempo necessário para a carga dos dados e a utilização de espaço em disco (implementação própria);

**4.** [prepare_queimadas.sh](https://github.com/vconrado/cap-386/blob/master/project/src/scripts/prepare_queimadas.sh): utiliza os programas anteriores para preparar automaticamente o arquivo para ser carregado no SciDB;

**5.** [test_rand.sh](https://github.com/vconrado/cap-386/blob/master/project/src/scripts/test_rand.sh): gera pontos aleatórios no domínio do cubo de dados e verifica se os dados armazenados no SciDB estão de acordo com os dados dos arquivos brutos (tif). Esse script foi feito para validar os programas implementados.


Demais scripts utilizados para a carga de dados podem se encontrados [aqui](https://github.com/vconrado/cap-386/tree/master/project/src). 

## 4. Disponibilização dos dados através de WTSS

Os arrays carregados no SciDB foram disponibilizados através de um serviço [WTSS](https://github.com/e-sensing/eows/blob/master/doc/wtss.md). 

Os serviços podem ser acessados em *links omitidos*. 

## 5.  Cliente Python para acesso aos dados de queimadas em Web Feature Service (WFS) 


O banco de focos de queimadas foi reestruturado e carregado em um servidor PostgreSQL. Na sequência, um servidor Geoserver foi configurado para servir esses dados via WFS. Para facilitar o acesso aos dados, foi criado um cliente em python para acessos aos dados. Esse cliente está disponível [aqui](https://github.com/vconrado/bdq.py).



## Proposta de Análise



- Para cada foco dentro dos talhões 
    - Obter dados (pixel) de Temperatura, Umidade, Risco de Fogo e Precipitação para o dia e X dias anteriores
    - Obter últimos X dados para bandas Landsat, índices (NDVI, NBRL, DNDVI, DNBRL)
- Para cada foco de queimada, obter outro ponto aleatoriamente dentro dos talhões que não é foco e obter mesmos dados

- ???? EDA/Classificação